# Count-5: patient count <= claim count

Description: check if all the patient counts are less or equal to transaction/claim counts.

Starting Author: Amy Jin (amy@careset.com)

Date: 07/23/2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [23]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [30]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) clm_cnt:                claim count column
# 4) bene_cnt:               patient count column
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:            PASS/FAIL
# 2) If FAIL, the test will also print out all the failed claim counts and patient counts.


def count_5(db_name, table_name, clm_cnt, bene_cnt):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        query = ('''
                SELECT *
                FROM {db}.{t1}
                WHERE {col1} < {col2}
                LIMIT 1;
        '''.format(db = db_name, t1 = table_name, col1 = clm_cnt, col2 = bene_cnt))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))

        if not len(rows):
            print ("Count-5 result for {}.{} : PASS".format(db_name, table_name))
        else:
            print ("Count-5 result for {}.{} : FAIL".format(db_name, table_name) + '\n')
            print ("The following {} & {} fail the test:".format(clm_cnt, bene_cnt) + '\n')
            # Use `DISTINCT` to reduce the calculation load
            query = ('''
                SELECT {col1}, {col2}
                FROM {db}.{t1}
                WHERE {col1} < {col2};
            '''.format(db = db_name, t1 = table_name, col1 = clm_cnt, col2 = bene_cnt))

            cur.execute(query)
            
            for row in cur.fetchall():
                for i in range(0,len(row)):
                    print (str(row[i]), end=", ")
                print ('\n')
            
        cur.close()
        connection.close()            

## Test Example

In [31]:
count_5('_amy', 'test_data_good', 'cnt_clm_id','cnt_bene_id')

Count-5 result for _amy.test_data_good : PASS


In [32]:
count_5('_amy', 'test_data_bad1', 'cnt_clm_id','cnt_bene_id')

Count-5 result for _amy.test_data_bad1 : FAIL

The following cnt_clm_id & cnt_clm_id fail the test:

5.0, 6, 

2.0, 5, 

4.0, 10, 

7.0, 9, 

6.0, 48, 

0.0, 7, 

3.0, 8, 

1.0, 6, 

0.0, 3, 

0.0, 5, 

0.0, 8, 

2.0, 10, 

3.0, 10, 

5.0, 9, 

8.0, 9, 

3.0, 7, 

3.0, 10, 

1.0, 18, 

4.0, 5, 

5.0, 8, 

0.0, 3, 

6.0, 10, 

3.0, 4, 

0.0, 1, 

0.0, 10, 

2.0, 5, 

3.0, 5, 

6.0, 9, 

11.0, 46, 

4.0, 38, 

3.0, 6, 

5.0, 9, 

4.0, 7, 

9.0, 10, 

3.0, 7, 

4.0, 8, 

10.0, 57, 

7.0, 9, 

2.0, 3, 

5.0, 9, 

13.0, 18, 

6.0, 9, 

0.0, 1, 

4.0, 7, 

2.0, 10, 

3.0, 7, 

3.0, 7, 

6.0, 10, 

1.0, 17, 

0.0, 7, 

0.0, 1, 

0.0, 2, 

3.0, 4, 

4.0, 17, 

5.0, 17, 

8.0, 10, 

2.0, 5, 

9.0, 28, 

3.0, 5, 

4.0, 5, 

3.0, 4, 

0.0, 6, 

1.0, 9, 

7.0, 17, 

0.0, 2, 

8.0, 17, 

6.0, 10, 

3.0, 9, 

4.0, 7, 

0.0, 5, 

7.0, 16, 

0.0, 8, 

8.0, 10, 

7.0, 8, 

2.0, 10, 

0.0, 6, 

5.0, 28, 

0.0, 4, 

9.0, 10, 

3.0, 5, 

0.0, 1, 

0.0, 4, 

7.0, 8, 

0.0, 6, 

7.0, 8, 

6.0, 8, 

0.0, 17,